# DWI preprocessing
## using MRTRIX, FSL and ANTS
### by Michael Paquette
#### Notes from Apr 7th

In this serie of bash notebook, I will be describing a basic DWI data preprocessing pipeline.

I will try to stick to this (loose) general format to describe each step:  

- What is the artefact we are trying to correct or the transformation we are trying to achieve  
- Why does this happen  
- What would happen if we didnt correct/modify 
- Why is this step here (somewhat arbitrary or specific ordering)  
- Important parameters of the tool (and which one are likely to need finetuning)  
- What to look for when doing QC (quality control)

In [1]:
# Like last time, we setup the basic variable with the folders
ROOTDIR='/data/pt_02586/'
SUBJECTDIR=$ROOTDIR'sub_01/'
# and we move to the preprocessing folder
cd $SUBJECTDIR'preprocessing'
pwd

# Like last time, we setup the basic variable with the folders
ROOTDIR='/data/pt_02586/'
SUBJECTDIR=$ROOTDIR'sub_01/'
# and we move to the preprocessing folder
cd $SUBJECTDIR'preprocessing'
pwd
/data/pt_02586/sub_01/preprocessing


: 1

### Quick recap from March 24th

We looked at the pratical example of tracking with 3 ROIs.  

1. Launched 3 tractography with tckgen  
- different ROI as seed for each  
- Allows for equal number of seeds per ROI even with diffrent volume 
- We use only a basic tractography mask, no include/exclude mask

2. Extract the bundles with tckedit  
- We now apply the include/exclude mask (i.e. the ROIs)  
- We have 3 pairs of ROIs, giving us 6 bundles  

3. Combine the bundles with tckedit  
- We recombine the bundles from same ROI pair
- We also build a bigger bundle with all.  

4. Compute track density image (TDI) with tckmap  
- We compute the TDI on the full bundle, usefull for some viz and QC

5. Project the FA on tracks with tcksample  
- We project the FA map on the streamlines points and average per streamline  


In [ ]:
# 1. Tractography with each SEED ROI

tckgen fod_lmax6.nii.gz tracto/prob_seed_ifgop_large.tck \
  -algorithm iFOD2 \
  -step 1.0 \
  -angle 45 \
  -minlength 30 \
  -maxlength 250 \
  -seeds 1e7 \
  -seed_image ifgop_peak_dilx2_warped_fa_th0p2_bin0p25.nii.gz \
  -mask fa_eddy_th0p2.nii.gz \
  -nthreads 40

tckgen fod_lmax6.nii.gz tracto/prob_seed_precg_large.tck \
  -algorithm iFOD2 \
  -step 1.0 \
  -angle 45 \
  -minlength 30 \
  -maxlength 250 \
  -seeds 1e7 \
  -seed_image precg_peak_dilx2_warped_fa_th0p2_bin0p25.nii.gz \
  -mask fa_eddy_th0p2.nii.gz \
  -nthreads 40

tckgen fod_lmax6.nii.gz tracto/prob_seed_ptc_large.tck \
  -algorithm iFOD2 \
  -step 1.0 \
  -angle 45 \
  -minlength 30 \
  -maxlength 250 \
  -seeds 1e7 \
  -seed_image ptc_peak_dilx2_warped_fa_th0p2_bin0p25.nii.gz \
  -mask fa_eddy_th0p2.nii.gz \
  -nthreads 40



In [ ]:
# 2. extract each bundle for each SEED 

tckedit tracto/prob_seed_ifgop_large.tck \
  tracto/prob_ifgop_precg_large.tck \
  -include ifgop_peak_dilx2_warped_fa_th0p2_bin0p25.nii.gz \
  -include precg_peak_dilx2_warped_fa_th0p2_bin0p25.nii.gz \
  -exclude ptc_peak_dilx2_warped_fa_th0p2_bin0p25.nii.gz \
  -nthreads 40

tckedit tracto/prob_seed_ifgop_large.tck \
  tracto/prob_ifgop_ptc_large.tck \
  -include ifgop_peak_dilx2_warped_fa_th0p2_bin0p25.nii.gz \
  -include ptc_peak_dilx2_warped_fa_th0p2_bin0p25.nii.gz \
  -exclude precg_peak_dilx2_warped_fa_th0p2_bin0p25.nii.gz \
  -nthreads 40

tckedit tracto/prob_seed_precg_large.tck \
  tracto/prob_precg_ptc_large.tck \
  -include precg_peak_dilx2_warped_fa_th0p2_bin0p25.nii.gz \
  -include ptc_peak_dilx2_warped_fa_th0p2_bin0p25.nii.gz \
  -exclude ifgop_peak_dilx2_warped_fa_th0p2_bin0p25.nii.gz \
  -nthreads 40

tckedit tracto/prob_seed_precg_large.tck \
  tracto/prob_precg_ifgop_large.tck \
  -include ptc_peak_dilx2_warped_fa_th0p2_bin0p25.nii.gz \
  -include ifgop_peak_dilx2_warped_fa_th0p2_bin0p25.nii.gz \
  -exclude precg_peak_dilx2_warped_fa_th0p2_bin0p25.nii.gz \
  -nthreads 40

tckedit tracto/prob_seed_ptc_large.tck \
  tracto/prob_ptc_ifgop_large.tck \
  -include ptc_peak_dilx2_warped_fa_th0p2_bin0p25.nii.gz \
  -include ifgop_peak_dilx2_warped_fa_th0p2_bin0p25.nii.gz \
  -exclude precg_peak_dilx2_warped_fa_th0p2_bin0p25.nii.gz \
  -nthreads 40

tckedit tracto/prob_seed_ptc_large.tck \
  tracto/prob_ptc_precg_large.tck \
  -include ptc_peak_dilx2_warped_fa_th0p2_bin0p25.nii.gz \
  -include precg_peak_dilx2_warped_fa_th0p2_bin0p25.nii.gz \
  -exclude ifgop_peak_dilx2_warped_fa_th0p2_bin0p25.nii.gz \
  -nthreads 40


In [ ]:
# 3. Concatenate bundles

tckedit tracto/prob_ifgop_precg_large.tck \
  tracto/prob_precg_ifgop_large.tck \
  tracto/prob_bundle_precg_ifgop_large.tck \
  -nthreads 40

tckedit tracto/prob_ifgop_ptc_large.tck \
  tracto/prob_ptc_ifgop_large.tck \
  tracto/prob_bundle_ptc_ifgop_large.tck \
  -nthreads 40

tckedit tracto/prob_precg_ptc_large.tck \
  tracto/prob_ptc_precg_large.tck \
  tracto/prob_bundle_ptc_precg_large.tck \
  -nthreads 40


# concatenate all
tckedit tracto/prob_ifgop_precg_large.tck \
  tracto/prob_precg_ifgop_large.tck \
  tracto/prob_ifgop_ptc_large.tck \
  tracto/prob_ptc_ifgop_large.tck \
  tracto/prob_precg_ptc_large.tck \
  tracto/prob_ptc_precg_large.tck \
  tracto/prob_bundles_large.tck \
  -nthreads 40

In [ ]:
# 4. TDI map

tckmap tracto/prob_bundles_large.tck tdi_prob_large.nii.gz \
  -template fa_eddy_th0p2.nii.gz \
  -contrast length

# colored with orientation version
tckmap tracto/prob_bundles_large.tck tdi_dec_prob_large.nii.gz \
  -template fa_eddy_th0p2.nii.gz \
  -dec \
  -contrast length

In [ ]:
# 5. Project FA on streamlines

tcksample tracto/prob_bundle_precg_ifgop_large_clean0p6.tck \
  fa_post_eddy.nii.gz \
  prob_bundle_precg_ifgop_large_clean0p6_meanFA.txt \
  -stat_tck mean

tcksample tracto/prob_bundle_ptc_ifgop_large_clean0p6.tck \
  fa_post_eddy.nii.gz \
  prob_bundle_ptc_ifgop_large_clean0p6_meanFA.txt \
  -stat_tck mean

tcksample tracto/prob_bundle_ptc_precg_large_clean0p6.tck \
  fa_post_eddy.nii.gz \
  prob_bundle_ptc_precg_large_clean0p6_meanFA.txt \
  -stat_tck mean

### Actual Connectome

The previous method is suitable for this small task with only 3 ROIS.  
We can "afford" to "manually" extract each bundle.  
To get the streamline count without using something like python,  
we STILL need to run something like tckinfo on each and note the counts.  

However, in connectomics, we often have a full parcellation of the cortex.  
This can easily be 150 ROIs per hemisphere. 
#### We need a systematic way to compute this matrix
- Compute a labelmap image  
- Combine tck, labelmap to compute trackcount connectome matrix  
- Add metrics like length or FA  

### tck2connectome
https://mrtrix.readthedocs.io/en/latest/reference/commands/tck2connectome.html  
https://mrtrix.readthedocs.io/en/dev/quantitative_structural_connectivity/structural_connectome.html  
https://mrtrix.readthedocs.io/en/dev/quantitative_structural_connectivity/labelconvert_tutorial.html#labelconvert-tutorial  

Tools for building connectome from streamlines and label map.  

The links give information to build label map easier from other type of data, such as freesurfer cortex parcellation.  

In [11]:
tck2connectome -h | cat

tck2connectome -h | cat
MRtrix 3.0.0                     tck2connectome                      Apr 23 2020

     tck2connectome: part of the MRtrix3 package

SYNOPSIS

     Generate a connectome matrix from a streamlines file and a node
     parcellation image

USAGE

     tck2connectome [ options ] tracks_in nodes_in connectome_out

        tracks_in    the input track file

        nodes_in     the input node parcellation image

        connectome_out  the output .csv file containing edge weights


EXAMPLE USAGES

     Default usage:
       $ tck2connectome tracks.tck nodes.mif connectome.csv -tck_weights_in weights.csv -out_assignments assignments.txt
     By default, the metric of connectivity quantified in the connectome matrix
     is the number of streamlines; or, if tcksift2 is used, the sum of
     streamline weights via the -tck_weights_in option. Use of the
     -out_assignments option is recommended as this enables subsequent use of
     the connectome2tck command.

     Gene

: 1

In [ ]:
# Make label map manually
# list of paths to individual ROI mask
# ASSUMES non overlapping mask

# KEEP this, maybe saved to a file, to "remember" ordering/mapping
LABELROI=(path/to/mask1.nii.gz \
          path/to/mask2.nii.gz \
          path/to/mask3.nii.gz)

# make empty label map 
mrcalc ${LABELROI[1]} 0 -mult labels.nii.gz

# initialise ROI index
ROIIDX=1

# for each roimask
for ROI in "${LABELROI[@]}"
do
    echo $ROI' index = '$ROIIDX;
    # create a map of 0s and ROIIDX, add it to current labels
    mrcalc $ROI $ROIIDX 0 -if labels.nii.gz -add labels.nii.gz -force;
    # increment
    ROIIDX=$((ROIIDX + 1))
done


### Testing time